In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from datetime import datetime
import re

In [42]:
def setup_driver():
    """
    Configure et initialise le driver Selenium
    """
    chrome_options = Options()
    # Commenter la ligne suivante pour voir le navigateur (utile pour debug)
    # chrome_options.add_argument('--headless')  
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--disable-gpu')
    
    # Options pour éviter la détection
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Masquer l'indicateur webdriver
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    return driver

In [43]:
def wait_for_element(driver, by, value, timeout=10):
    """
    Attend qu'un élément soit présent sur la page
    """
    try:
        element = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((by, value))
        )
        return element
    except:
        return None

In [44]:
def extract_text_safe(driver, selector, by=By.CSS_SELECTOR):
    """
    Extrait le texte d'un élément de manière sécurisée
    """
    try:
        element = driver.find_element(by, selector)
        return element.text.strip() if element else 'N/A'
    except:
        return 'N/A'

In [ ]:
def extract_car_specs_automobile_tn(driver, specs_url):
    """
    Extrait les spécifications techniques d'une voiture depuis la fiche technique
    """
    try:
        print(f"    📋 Extraction fiche technique...")
        driver.get(specs_url)
        time.sleep(3)
        
        specs_data = {}
        
        # Liste des caractéristiques à extraire
        target_specs = [
            'Disponibilité',
            'Carrosserie',
            'Garantie',
            'Nombre de places',
            'Nombre de portes',
            'Nombre de cylindres',
            'Energie',
            'Puissance fiscale',
            'Couple',
            'Boîte',
            'Nombre de rapports',
            'Consommation urbaine',
            'Consommation extra-urbaine'
        ]
        
        # Extraire toutes les tables de spécifications
        tables = driver.find_elements(By.CSS_SELECTOR, 'table')
        
        spec_count = 0
        for table in tables:
            try:
                rows = table.find_elements(By.TAG_NAME, 'tr')
                for row in rows:
                    try:
                        th = row.find_element(By.TAG_NAME, 'th')
                        td = row.find_element(By.TAG_NAME, 'td')
                        
                        key = th.text.strip()
                        value = td.text.strip()
                        
                        # Extraire uniquement les caractéristiques ciblées
                        if key in target_specs and value:
                            specs_data[key] = value
                            spec_count += 1
                            print(f"      ✓ {key}: {value}")
                    except:
                        continue
            except:
                continue
        
        print(f"    ✓ {spec_count} spécifications extraites")
        return specs_data
        
    except Exception as e:
        print(f"    ❌ Erreur extraction specs: {e}")
        return {}

In [50]:
def extract_version_details_automobile_tn(driver, model_url):
    """
    Extrait les détails de toutes les versions d'un modèle
    """
    try:
        print(f"  📄 Chargement du modèle...")
        driver.get(model_url)
        time.sleep(3)
        
        versions_data = []
        
        # Extraire le nom du modèle
        try:
            model_name_elem = driver.find_element(By.CSS_SELECTOR, 'h1, .bloc-title h3')
            model_name = model_name_elem.text.strip()
        except:
            # Extraire depuis l'URL
            model_name = model_url.split('/')[-1].replace('-', ' ').title()
        
        print(f"  🚗 Modèle: {model_name}")
        
        # Vérifier si on est directement sur une fiche technique (pas de tableau de versions)
        version_rows = driver.find_elements(By.CSS_SELECTOR, 'table.versions tbody tr')
        
        if not version_rows or len(version_rows) == 0:
            print(f"  ℹ️ Pas de tableau de versions - extraction directe de la fiche technique")
            
            # Extraire directement les données depuis la page actuelle
            car_data = {}
            car_data['Modèle'] = model_name
            car_data['Version'] = 'Version unique'
            
            # Extraire le prix si présent
            try:
                price_elem = driver.find_element(By.CSS_SELECTOR, '.price, .prix, span[class*="price"]')
                car_data['Prix'] = price_elem.text.strip()
                print(f"    💰 Prix: {car_data['Prix']}")
            except:
                car_data['Prix'] = 'N/A'
                print(f"    ⚠️ Prix non disponible")
            
            car_data['Promo'] = 'Non'
            
            # Extraire les spécifications depuis la page actuelle
            specs_data = extract_car_specs_automobile_tn(driver, model_url)
            car_data.update(specs_data)
            car_data['URL'] = model_url
            
            versions_data.append(car_data)
            print(f"    ✅ Fiche technique extraite")
            
        else:
            # Tableau de versions présent - extraction normale
            print(f"  📊 {len(version_rows)} version(s) trouvée(s)")
            print(f"  ℹ️ Extraction de la première version uniquement")
            
            # Extraire seulement la première version
            for idx, row in enumerate(version_rows[:1], 1):
                try:
                    car_data = {}
                    car_data['Modèle'] = model_name
                    specs_url = model_url  # Initialiser avec l'URL du modèle par défaut
                    
                    # Extraire le nom de la version
                    version_link = row.find_element(By.CSS_SELECTOR, 'td.version a')
                    version_name = version_link.text.strip()
                    car_data['Version'] = version_name
                    
                    print(f"    [{idx}] {version_name}")
                    
                    # Extraire le prix
                    try:
                        price_elem = row.find_element(By.CSS_SELECTOR, 'td.price')
                        price_text = price_elem.text.strip()
                        car_data['Prix'] = price_text
                        print(f"      💰 Prix: {price_text}")
                    except Exception as e:
                        car_data['Prix'] = 'N/A'
                        print(f"      ⚠️ Prix non trouvé: {e}")
                    
                    # Vérifier si promo
                    try:
                        promo_badge = row.find_element(By.CSS_SELECTOR, 'span.badge.promo')
                        car_data['Promo'] = 'Oui'
                        
                        # Extraire ancien prix si disponible
                        try:
                            old_price = row.find_element(By.CSS_SELECTOR, 'i.text-muted s')
                            car_data['Ancien Prix'] = old_price.text.strip()
                            print(f"      🏷️ Promo - Ancien prix: {car_data['Ancien Prix']}")
                        except:
                            pass
                    except:
                        car_data['Promo'] = 'Non'
                    
                    # Extraire le lien vers la fiche technique
                    try:
                        specs_link_elem = row.find_element(By.CSS_SELECTOR, 'td.specs a')
                        specs_url = specs_link_elem.get_attribute('href')
                        
                        if specs_url:
                            # Extraire les spécifications
                            specs_data = extract_car_specs_automobile_tn(driver, specs_url)
                            car_data.update(specs_data)
                            
                            # Retourner à la page du modèle
                            driver.get(model_url)
                            time.sleep(2)
                        else:
                            print(f"      ⚠️ Lien fiche technique vide")
                        
                    except Exception as e:
                        print(f"      ⚠️ Pas de fiche technique disponible: {e}")
                    
                    # URL de la version
                    car_data['URL'] = specs_url
                    
                    versions_data.append(car_data)
                    print(f"      ✅ Version extraite")
                    
                except Exception as e:
                    print(f"      ❌ Erreur version {idx}: {e}")
                    import traceback
                    traceback.print_exc()
                    continue
        
        print(f"\n  ✅ Total: {len(versions_data)} version(s) extraite(s) pour ce modèle")
        return versions_data
        
    except Exception as e:
        print(f"  ❌ Erreur générale: {e}")
        import traceback
        traceback.print_exc()
        return []

In [51]:
def get_car_model_links_automobile_tn(driver, search_url, start_page=1, max_pages=1, max_cars=None):
    """
    Récupère les liens vers les pages de modèles depuis la page de recherche avec pagination
    start_page: page de départ (par défaut 1)
    max_pages: nombre de pages à scraper
    max_cars: nombre maximum de modèles à récupérer (None = tous)
    """
    all_model_links = []
    
    try:
        end_page = start_page + max_pages - 1
        print(f"🔍 Récupération des liens depuis les pages {start_page} à {end_page}...")
        if max_cars:
            print(f"   Limite: {max_cars} modèles maximum")
        
        for page_num in range(start_page, end_page + 1):
            # Vérifier si on a atteint la limite
            if max_cars and len(all_model_links) >= max_cars:
                print(f"\n✅ Limite de {max_cars} modèles atteinte")
                break
            
            # Construire l'URL de la page
            if page_num == 1:
                page_url = search_url
            else:
                page_url = f"{search_url}?page={page_num}"
            
            print(f"\n📄 Page {page_num}: {page_url}")
            
            try:
                driver.get(page_url)
                time.sleep(4)
                
                # Scroll pour charger tous les éléments
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                
                # Trouver tous les articles avec data-key
                articles = driver.find_elements(By.CSS_SELECTOR, 'div.articles span[data-key]')
                print(f"  📊 {len(articles)} modèle(s) trouvé(s) sur cette page")
                
                page_links = []
                for article in articles:
                    # Vérifier si on a atteint la limite
                    if max_cars and len(all_model_links) >= max_cars:
                        break
                        
                    try:
                        link_elem = article.find_element(By.CSS_SELECTOR, 'a[href*="/fr/neuf/"]')
                        href = link_elem.get_attribute('href')
                        
                        if href and href not in all_model_links:
                            all_model_links.append(href)
                            page_links.append(href)
                            
                            # Extraire le nom du modèle pour affichage
                            try:
                                model_title = link_elem.find_element(By.TAG_NAME, 'h2')
                                print(f"    ✓ {model_title.text.strip()}")
                            except:
                                print(f"    ✓ Lien trouvé")
                                
                    except:
                        continue
                
                print(f"  ✓ {len(page_links)} nouveau(x) modèle(s) sur cette page")
                print(f"  📊 Total actuel: {len(all_model_links)} modèles")
                
            except Exception as e:
                print(f"  ❌ Erreur sur la page {page_num}: {e}")
                continue
        
        print(f"\n✅ Total: {len(all_model_links)} modèle(s) à traiter")
        return all_model_links
        
    except Exception as e:
        print(f"❌ Erreur générale: {e}")
        import traceback
        traceback.print_exc()
        return all_model_links

In [52]:
def scrape_automobile_tn_selenium(start_page=1, max_pages=1, max_cars=None):
    """
    Fonction principale pour scraper automobile.tn
    start_page: page de départ (par défaut 1)
    max_pages: nombre de pages à scraper
    max_cars: nombre maximum de modèles à scraper (None = tous)
    """
    print(f"{'='*60}")
    print(f"🚀 SCRAPING AUTOMOBILE.TN - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{'='*60}")
    print(f"Pages: {start_page} à {start_page + max_pages - 1}")
    if max_cars:
        print(f"Limite: {max_cars} modèle(s)")
    print()
    
    driver = None
    all_cars_data = []
    
    try:
        # Initialiser le driver
        print("🔧 Configuration du WebDriver...")
        driver = setup_driver()
        print("✓ WebDriver configuré\n")
        
        search_url = "https://www.automobile.tn/fr/neuf/recherche/s="
        
        # Récupérer les liens des modèles
        model_links = get_car_model_links_automobile_tn(driver, search_url, start_page, max_pages, max_cars)
        
        if not model_links:
            print("❌ Aucun modèle trouvé")
            return pd.DataFrame()
        
        # Traiter chaque modèle
        for i, model_url in enumerate(model_links, 1):
            print(f"\n{'='*60}")
            print(f"[{i}/{len(model_links)}] TRAITEMENT: {model_url}")
            print(f"{'='*60}")
            
            versions = extract_version_details_automobile_tn(driver, model_url)
            
            if versions:
                all_cars_data.extend(versions)
                print(f"\n  ✅ {len(versions)} version(s) ajoutée(s)")
            else:
                print(f"\n  ❌ Aucune version extraite")
            
            # Pause entre les modèles
            if i < len(model_links):
                print(f"\n  ⏸️ Pause de 3 secondes...")
                time.sleep(3)
        
    except Exception as e:
        print(f"\n❌ Erreur générale: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        # Fermer le driver
        if driver:
            driver.quit()
            print("\n🔒 WebDriver fermé")
    
    # Créer le DataFrame
    if all_cars_data:
        df = pd.DataFrame(all_cars_data)
        print(f"\n{'='*60}")
        print(f"✅ SCRAPING TERMINÉ!")
        print(f"{'='*60}")
        print(f"Total: {len(all_cars_data)} voiture(s) extraite(s)")
        print(f"Colonnes: {len(df.columns)}")
        return df
    else:
        print("\n❌ Aucune donnée extraite")
        return pd.DataFrame()

In [53]:
# Scraper plusieurs pages pour obtenir 40 modèles
df_voitures = scrape_automobile_tn_selenium(start_page=1, max_pages=5, max_cars=40)

🚀 SCRAPING AUTOMOBILE.TN - 2025-11-28 14:28:47
Pages: 1 à 5
Limite: 40 modèle(s)

🔧 Configuration du WebDriver...
✓ WebDriver configuré

🔍 Récupération des liens depuis les pages 1 à 5...
   Limite: 40 modèles maximum

📄 Page 1: https://www.automobile.tn/fr/neuf/recherche/s=
✓ WebDriver configuré

🔍 Récupération des liens depuis les pages 1 à 5...
   Limite: 40 modèles maximum

📄 Page 1: https://www.automobile.tn/fr/neuf/recherche/s=
  📊 24 modèle(s) trouvé(s) sur cette page
    ✓ SKODA FABIA
    ✓ SKODA KUSHAQ
    ✓ SKODA SCALA
    ✓ SKODA KAMIQ
    ✓ SKODA OCTAVIA
    ✓ SKODA SUPERB
    ✓ AVANTIER AVANTIER C
    ✓ SUZUKI CELERIO POPULAIRE
    ✓ RENAULT KWID POPULAIRE
  📊 24 modèle(s) trouvé(s) sur cette page
    ✓ SKODA FABIA
    ✓ SKODA KUSHAQ
    ✓ SKODA SCALA
    ✓ SKODA KAMIQ
    ✓ SKODA OCTAVIA
    ✓ SKODA SUPERB
    ✓ AVANTIER AVANTIER C
    ✓ SUZUKI CELERIO POPULAIRE
    ✓ RENAULT KWID POPULAIRE
    ✓ KIA PICANTO POPULAIRE
    ✓ CENNTRO LOGISTAR 100
    ✓ DFSK K01S
    ✓ CITRO

In [54]:
# Afficher les résultats
if not df_voitures.empty:
    print("\n" + "="*100)
    print("RÉSULTATS DU SCRAPING")
    print("="*100)
    display(df_voitures)
    
    print("\n" + "="*100)
    print("INFORMATIONS SUR LE DATASET")
    print("="*100)
    print(f"Nombre de voitures: {len(df_voitures)}")
    print(f"Nombre de colonnes: {len(df_voitures.columns)}")
    print(f"\nColonnes extraites:")
    for col in df_voitures.columns:
        print(f"  • {col}")
else:
    print("\n❌ Aucune donnée à afficher")


RÉSULTATS DU SCRAPING


,Modèle,Version,Prix,Promo,Ancien Prix,DISPONIBILITÉ,CARROSSERIE,GARANTIE,NOMBRE DE PLACES,NOMBRE DE PORTES,...,KIT SÉCURITÉ,SYSTÈME DE DESCENTE EN CÔTE,ORDINATEUR DE BORD,COFFRE À BAGAGES,TEMPS DE RECHARGE RAPIDE (DC),CONTRÔLE DE PRESSION DES PNEUS,ANTIBROUILLARDS,AUTONOMIE ÉLECTRIQUE (NEDC),RÉGULATEUR DE VITESSE,TAPIS DE SOL
0,SKODA FABIA,1.0 L MPI Essence,66 980 DT,Oui,68.980 DT,Disponible,Citadine,3 ans,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SKODA KUSHAQ,1.0 TSI Ambition,78 980 DT,Non,NaN,Sur arrivage,SUV,3 ans,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SKODA SCALA,1.0 L TSI Essence DSG,86 980 DT,Oui,91.980 DT,Sur arrivage,Compacte,3 ans,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SKODA KAMIQ,1.0 L TSI Essence DSG,89 980 DT,Oui,94.980 DT,Sur arrivage,SUV,3 ans,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SKODA OCTAVIA,1.4 TSI Essence BVA,114 980 DT,Oui,124.980 DT,Disponible,Berline,3 ans,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SKODA SUPERB,1.5 TSI Essence DSG,155 980 DT,Non,NaN,En pré-commande,Berline,3 ans,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AVANTIER AVANTIER C\n14 KWH,Version unique,à partir de 32 800 DT,Non,NaN,En pré-commande,Utilitaire,5 ans ou 100000 km | Batterie garantie 8 ans o...,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SUZUKI CELERIO POPULAIRE\n1.0 L GL,Version unique,à partir de 32 190 DT,Non,NaN,Disponible | Sous réserve d'acceptation du dos...,Citadine,3 ans,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,RENAULT KWID POPULAIRE\n1.0 L,Version unique,N/A,Non,NaN,Sous réserve d'acceptation du dossier par le m...,SUV,2 ans,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,KIA PICANTO POPULAIRE\n1.0 L,Version unique,à partir de 28 935 DT,Non,NaN,Disponible | Sous réserve d'acceptation du dos...,Citadine,5 ans,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



INFORMATIONS SUR LE DATASET
Nombre de voitures: 40
Nombre de colonnes: 93

Colonnes extraites:
  • Modèle
  • Version
  • Prix
  • Promo
  • Ancien Prix
  • DISPONIBILITÉ
  • CARROSSERIE
  • GARANTIE
  • NOMBRE DE PLACES
  • NOMBRE DE PORTES
  • NOMBRE DE CYLINDRES
  • ENERGIE
  • PUISSANCE FISCALE
  • PUISSANCE (CH.DIN)
  • COUPLE
  • CYLINDRÉE
  • BOÎTE
  • NOMBRE DE RAPPORTS
  • TRANSMISSION
  • LONGUEUR
  • LARGEUR
  • HAUTEUR
  • VOLUME DU COFFRE
  • 0-100 KM/H
  • VITESSE MAXI
  • CONSOMMATION URBAINE
  • CONSOMMATION EXTRA-URBAINE
  • CONSOMMATION MIXTE
  • EMISSIONS DE CO2
  • AIRBAGS
  • ANTI-PATINAGE
  • BARRE STABILISATRICE
  • CALANDRE
  • ELÉMENTS EXTÉRIEURS COULEUR CARROSSERIE
  • FEUX À LED
  • JANTES
  • PHARES
  • AUTORADIO
  • CONNECTIVITÉ
  • ECRAN CENTRAL
  • ACCOUDOIRS
  • APPUIS TÊTES ARRIÈRES
  • KIT RANGEMENT
  • POMMEAU DE LEVIER DE VITESSE
  • SELLERIE
  • SIÈGES RÉGLABLES EN HAUTEUR
  • VITRES
  • VOLANT
  • VOLANT RÉGLABLE
  • CLIMATISATION
  • FERMETURE CE

In [55]:
# Afficher les détails de la première voiture
if not df_voitures.empty:
    print("\n" + "="*100)
    print("DÉTAILS DE LA PREMIÈRE VOITURE")
    print("="*100)
    for col, value in df_voitures.iloc[7].items():
        print(f"{col:30s}: {value}")


DÉTAILS DE LA PREMIÈRE VOITURE
Modèle                        : SUZUKI CELERIO POPULAIRE
1.0 L GL
Version                       : Version unique
Prix                          : à partir de 32 190 DT
Promo                         : Non
Ancien Prix                   : nan
DISPONIBILITÉ                 : Disponible | Sous réserve d'acceptation du dossier par le ministère du commerce
CARROSSERIE                   : Citadine
GARANTIE                      : 3 ans
NOMBRE DE PLACES              : 5
NOMBRE DE PORTES              : 5
NOMBRE DE CYLINDRES           : 3
ENERGIE                       : Essence
PUISSANCE FISCALE             : 4 CV
PUISSANCE (CH.DIN)            : 68 CH
COUPLE                        : 90 nm 3500 tr/min
CYLINDRÉE                     : 998 CM³
BOÎTE                         : Manuelle
NOMBRE DE RAPPORTS            : 5
TRANSMISSION                  : Traction
LONGUEUR                      : 3695 mm
LARGEUR                       : 1655 mm
HAUTEUR                       : 155

### Sauvegarde des données

In [56]:
# Sauvegarder les données en CSV et Excel
if not df_voitures.empty:
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # CSV
    csv_filename = f"automobile_tn_{timestamp}.csv"
    df_voitures.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"\n💾 Données sauvegardées dans: {csv_filename}")
    
    # Excel
    try:
        excel_filename = f"automobile_tn_{timestamp}.xlsx"
        df_voitures.to_excel(excel_filename, index=False, engine='openpyxl')
        print(f"💾 Données sauvegardées dans: {excel_filename}")
    except:
        print("⚠️ Installation d'openpyxl nécessaire pour Excel")
else:
    print("\n❌ Aucune donnée à sauvegarder")


💾 Données sauvegardées dans: automobile_tn_20251128_145956.csv
💾 Données sauvegardées dans: automobile_tn_20251128_145956.xlsx
